# RubbishClassifier CNN 网络架构可视化

本notebook展示了垃圾分类CNN模型的详细架构结构。

## 1. 完整网络架构图

下图展示了完整的CNN网络结构，包括所有层的连接关系和数据流：

In [ ]:
from IPython.display import Image, display
import matplotlib.pyplot as plt

# 显示完整网络架构图
display(Image('./network_architecture.png', width=1000))

## 2. 详细卷积块结构

下图详细展示了单个卷积块的内部结构：

In [ ]:
# 显示详细卷积块图
display(Image('./conv_block_detail.png', width=800))

## 3. 网络结构详细说明

### 输入层
- **输入尺寸**: 224×224×3 (RGB图像)
- **预处理**: letterbox缩放 + 归一化 [0,1]

### 卷积特征提取部分

#### 第1个卷积块
- **Conv1**: 3→32通道，3×3卷积核，SAME填充
- **BatchNorm1**: 32通道批量归一化
- **ReLU**: 激活函数
- **MaxPool**: 2×2池化，步长2
- **输出**: 112×112×32

#### 第2个卷积块
- **Conv2**: 32→64通道，3×3卷积核
- **BatchNorm2**: 64通道批量归一化  
- **ReLU + MaxPool**: 激活+池化
- **输出**: 56×56×64

#### 第3个卷积块
- **Conv3**: 64→128通道，3×3卷积核
- **BatchNorm3**: 128通道批量归一化
- **ReLU + MaxPool**: 激活+池化
- **输出**: 28×28×128

#### 第4个卷积块
- **Conv4**: 128→256通道，3×3卷积核
- **BatchNorm4**: 256通道批量归一化
- **ReLU + MaxPool**: 激活+池化
- **输出**: 14×14×256

#### 第5个卷积块
- **Conv5**: 256→512通道，3×3卷积核
- **BatchNorm5**: 512通道批量归一化
- **ReLU**: 激活函数（无池化）
- **输出**: 14×14×512

### 全局池化与分类部分

#### 全局平均池化
- **操作**: 对空间维度(H,W)求平均
- **输出**: 512维特征向量

#### 全连接分类器
- **FC1**: 512→256 + ReLU
- **FC2**: 256→128 + ReLU  
- **FC3**: 128→12 (输出logits)

### 最终输出
- **形状**: (batch_size, 12)
- **含义**: 12个垃圾分类类别的logits分数

## 4. 网络参数统计

In [ ]:
# 计算网络参数数量
def calculate_parameters():
    params = {}
    
    # 卷积层参数 (weight + bias)
    params['Conv1'] = (3*3*3*32) + 32
    params['Conv2'] = (3*3*32*64) + 64  
    params['Conv3'] = (3*3*64*128) + 128
    params['Conv4'] = (3*3*128*256) + 256
    params['Conv5'] = (3*3*256*512) + 512
    
    # 批量归一化参数 (scale + shift)
    params['BN1'] = 32 * 2
    params['BN2'] = 64 * 2  
    params['BN3'] = 128 * 2
    params['BN4'] = 256 * 2
    params['BN5'] = 512 * 2
    
    # 全连接层参数
    params['FC1'] = (512*256) + 256
    params['FC2'] = (256*128) + 128  
    params['FC3'] = (128*12) + 12
    
    return params

params = calculate_parameters()
total_params = sum(params.values())

print("网络参数统计:")
print("=" * 40)
for layer, count in params.items():
    print(f"{layer:<10}: {count:>10,} 参数")

print("=" * 40)
print(f"{'总计':<10}: {total_params:>10,} 参数")
print(f"约 {total_params/1e6:.2f}M 参数")

## 5. 特征图尺寸变化

In [ ]:
# 展示特征图尺寸变化
layers = [
    ("Input", (224, 224, 3)),
    ("Conv1+Pool", (112, 112, 32)),
    ("Conv2+Pool", (56, 56, 64)),
    ("Conv3+Pool", (28, 28, 128)),
    ("Conv4+Pool", (14, 14, 256)),
    ("Conv5", (14, 14, 512)),
    ("Global AvgPool", (512,)),
    ("FC1", (256,)),
    ("FC2", (128,)),
    ("Output", (12,))
]

print("特征图尺寸变化:")
print("=" * 50)
for i, (name, shape) in enumerate(layers):
    if len(shape) == 3:
        h, w, c = shape
        pixels = h * w * c
        print(f"{i+1:2d}. {name:<15}: {h:3d}×{w:3d}×{c:3d} = {pixels:>8,} 元素")
    else:
        c = shape[0]
        print(f"{i+1:2d}. {name:<15}: {c:>11} 元素")
    
    if i < len(layers) - 1:
        print("    ↓")

## 6. 12类垃圾分类类别

In [ ]:
# 显示分类类别
classes = [
    (0, "🔋 battery", "电池"),
    (1, "🥬 biological", "生物垃圾"), 
    (2, "🍺 brown-glass", "棕色玻璃"),
    (3, "📦 cardboard", "纸板"),
    (4, "👕 clothes", "衣物"),
    (5, "🍃 green-glass", "绿色玻璃"),
    (6, "🔧 metal", "金属"),
    (7, "📄 paper", "纸张"),
    (8, "♻️ plastic", "塑料"),
    (9, "👟 shoes", "鞋子"),
    (10, "🗑️ trash", "一般垃圾"),
    (11, "🥛 white-glass", "白色玻璃")
]

print("垃圾分类类别 (12类):")
print("=" * 60)
for idx, emoji_name, chinese in classes:
    print(f"类别 {idx:2d}: {emoji_name:<20} ({chinese})")

print(f"\n网络输出: 12维向量，每个元素对应一个类别的logits分数")
print(f"预测类别: argmax(logits) → 类别ID")

## 7. 网络设计特点

### 🎯 设计亮点

1. **渐进式特征提取**
   - 从低级特征(边缘、纹理)到高级特征(物体形状)
   - 通道数逐渐增加: 3→32→64→128→256→512

2. **批量归一化**
   - 每个卷积层后都有BatchNorm
   - 加速训练收敛，提高稳定性

3. **全局平均池化**
   - 替代传统的展平操作
   - 减少参数数量，防止过拟合

4. **适中的网络深度**
   - 5个卷积层平衡了特征提取能力和计算效率
   - 约1.7M参数，适合中等规模数据集

### 🚀 优化空间

1. **数据增强**: 旋转、翻转、色彩变换
2. **Dropout**: 在全连接层添加dropout防止过拟合
3. **学习率调度**: 使用cosine annealing等策略
4. **残差连接**: 考虑添加skip connections